In [ ]:
import os
import gc
import datasets, transformers

from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

os.environ["WANDB_DISABLED"] = "true"
path = '../input/'

In [ ]:
class CFG:
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = {
        
'group_1' : [f'{path}deberta-base-crazy-2/', f'{path}bert-for-patents-otabek/', f'{path}xlm-roberta-0904/', f'{path}deberta-v3-large-otabek/'],#, f'{path}deberta-large-9107/',],
# 'group_2' : [f'{path}bert-for-patent-5fold/', f'{path}deberta-large-v1/', f'{path}deberta-v3-5folds/'],
'group_3' : [f'{path}deberta-v3-large-0915/', f'{path}deberta-base-crazy-1/', f'{path}deberta-base-crazy-5/', f'{path}deberta-large-09917/', f'{path}xlm-roberta-otabek/', f'{path}bert-for-patents-2/'], 
'group_4' : [f'{path}deberta-v3-large-092/', f'{path}deberta-large-913/', f'{path}deberta-base-crazy-3/', f'{path}deberta-base-crazy-4/', f'{path}xml-roberta-0900/', f'{path}bert-for-patents-0917/'],
        
    } 
                    
#     model_num = 1
    num_fold = 4
#     num_group = 4
    
fold_dict = {'group_1': '_', 'group_2': '', 'group_3': '_','group_4': '_'}

In [ ]:
# Добавить веса как у Китайца

# Группа 1  => Катбуст + фичи  => бленд всех катбустов (20)

# https://www.kaggle.com/datasets/bulivington/deberta-base-crazy-2/
# https://www.kaggle.com/datasets/obatek/bert-for-patents-otabek
# https://www.kaggle.com/datasets/obatek/xlm-roberta-0904
# https://www.kaggle.com/datasets/obatek/deberta-v3-large-otabek
# https://www.kaggle.com/datasets/obatek/deberta-large-9107
    
# Группа 2  !!! Не смешивать !!!

# https://www.kaggle.com/datasets/luaubrey/bert-for-patent-5fold
# https://www.kaggle.com/datasets/luaubrey/deberta-large-v1
# https://www.kaggle.com/datasets/luaubrey/deberta-v3-5folds/

# Группа 3

# https://www.kaggle.com/datasets/obatek/deberta-v3-large-0915
# https://www.kaggle.com/datasets/bulivington/deberta-base-crazy-1/
# https://www.kaggle.com/datasets/bulivington/deberta-base-crazy-5/
# https://www.kaggle.com/datasets/obatek/deberta-large-09917
# https://www.kaggle.com/datasets/obatek/xlm-roberta-otabek
# https://www.kaggle.com/datasets/obatek/bert-for-patents-2
    
# Группа 4

# https://www.kaggle.com/datasets/obatek/deberta-v3-large-092
# https://www.kaggle.com/datasets/obatek/deberta-large-913
# https://www.kaggle.com/datasets/bulivington/deberta-base-crazy-3/
# https://www.kaggle.com/datasets/bulivington/deberta-base-crazy-4/
# https://www.kaggle.com/datasets/obatek/xml-roberta-0900
# https://www.kaggle.com/datasets/obatek/bert-for-patents-0917

In [ ]:
train = pd.read_csv("../input/uspppm-folds/updated_train_5_folds.csv", index_col = 0)
train.head(5)

In [ ]:
# train.id.value_counts()

In [ ]:
# weights = [0.5, 0.3, 0.2]

model_features = {}
for group in tqdm(CFG.model_path.keys()):  # Итерируемся по группам моделей
    
    print(CFG.model_path[group])
    for num in range(len(CFG.model_path[group])): # Итерируемся по моделям 
        
        
        
        def process_test(unit):
                return {
                **tokenizer( unit['input'], unit['target'])
            }

        def process_valid(unit):
            return {
            **tokenizer( unit['input'], unit['target']),
            'label': unit['score']
        }


        predictions = []
        scores = []
        for fold in range(CFG.num_fold):   # Формируем новый датасет из предиктов
            tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_path[group][num]}fold{fold_dict[group]}{fold}')            
            
            train_ds = datasets.Dataset.from_pandas(train[train.fold == fold])
            train_ds = train_ds.map(process_test, remove_columns=['id', 'target', 'input', '__index_level_0__'])

            model = AutoModelForSequenceClassification.from_pretrained(f'{CFG.model_path[group][num]}fold{fold_dict[group]}{fold}', num_labels=1)
            trainer = Trainer(model, tokenizer=tokenizer)
            outputs = trainer.predict(train_ds)
            
            prediction = outputs.predictions.reshape(-1).tolist() # * (weights[i] / 5)    
            score = train[train.fold == fold]['score'].values.tolist()
            
            predictions.extend(prediction)
            scores.extend(score)
            
            del tokenizer, model, train_ds; gc.collect()

        model_features[f'{group}_{num}'] = list(zip(predictions, scores))

del train

In [ ]:
dict_to_df = {}
for n, fs in enumerate(model_features.keys()):
    dict_to_df[fs] = [i[0] for i in model_features[fs]]
    scores = [i[1] for i in model_features[fs]]
    
#     context = [i[1] for i in model_features[fs]]
    
#     dict_to_df[f'{n}_score'] = scores
    
dict_to_df['score'] = scores

train_final = pd.DataFrame(dict_to_df)

del model_features
train_final.head()

In [ ]:
# scores_c = [c for c in train_final.columns if 'score' in c]
# train_final[scores_c].std()

In [ ]:
# train_final

# Blend for Catboost predictions

In [ ]:
from sklearn.model_selection import KFold # k-фолдная валидация
from catboost import Pool, CatBoostClassifier, CatBoostRegressor

class ProfitMetric:
    def is_max_optimal(self):
        return True # greater is better

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        y_pred = approxes[0]
        y_true = np.array(target)

        output_weight = 1 # weight is not used
 
        return np.corrcoef(y_pred, y_true)[0][1], output_weight

    def get_final_error(self, error, weight):
        return error

In [ ]:
n_splits = 3
print('n_fold', n_splits)
clfs = {}
targets = 'score'

# kFold_random_state = [42]# , 666, 228, 777, 2021, 75]
random_state = 42

N = len(CFG.model_path.keys())*n_splits

for group in CFG.model_path.keys():

    filtered_features = [col for col in train_final.columns if group in col]

    X = train_final[filtered_features]#.copy()
    y = train_final[targets]#.copy()

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    kfold = 0
    for train_index, test_index in kf.split(X):
    #     kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

        X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        train_dataset = Pool(data=X_train, label=y_train)
        eval_dataset = Pool(data=X_test, label=y_test)

        print(X_train.shape, y_train.shape)
        
        clf = CatBoostRegressor(random_state=42, n_estimators=30_000,
#                                 metric_period=100,
                                max_depth=3,
                                early_stopping_rounds=250,
                                eval_metric=ProfitMetric())
        
        if clfs.get(group, None) is not None:   
            clfs[group].append(clf)
        else:
            clfs[group] = [clf]
    
        # clf.fit(X_train, y_train, eval_set=(X_test, y_test),
        clf.fit(train_dataset, eval_set=eval_dataset, verbose = 500, use_best_model = True, plot = False)
        clf.save_model("cool_catboost_model_{}_kfold_{}".format(group, kfold))
        kfold+=1
        
# assert len(clfs) == N

In [ ]:
titles = pd.read_csv('../input/cpc-codes/titles.csv')

test = pd.read_csv(f"{CFG.input_path}test.csv")
test = test.merge(titles, left_on='context', right_on='code')
test['input'] = test['title']+'[SEP]'+test['anchor']
test = test.drop(columns=["context", "code", "class", "subclass", "group", "main_group", "anchor", "title", "section"])

# Test Inference

In [ ]:
predictions = []
# weights = [0.5, 0.3, 0.2]

model_features = {}
for group in CFG.model_path.keys():

    print(CFG.model_path[group])
    for num in range(len(CFG.model_path[group])): # Итерируемся по моделям 
        
        tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_path[group][num]}fold{fold_dict[group]}{fold}')

        def process_test(unit):
                return {
                **tokenizer( unit['input'], unit['target'])
            }

        def process_valid(unit):
            return {
            **tokenizer( unit['input'], unit['target']),
            'label': unit['score']
        }

        test_ds = datasets.Dataset.from_pandas(test)
        test_ds = test_ds.map(process_test, remove_columns=['id', 'target', 'input', '__index_level_0__'])

        
        for fold in range(CFG.num_fold):   # Формируем новый датасет из предиктов
            tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_path[group][num]}fold{fold_dict[group]}{fold}')
            
            model = AutoModelForSequenceClassification.from_pretrained(f'{CFG.model_path[group][num]}fold{fold_dict[group]}{fold}', num_labels=1)
            trainer = Trainer(model, tokenizer=tokenizer)
            outputs = trainer.predict(test_ds)
            
            prediction = outputs.predictions.reshape(-1).tolist() # * (weights[i] / 5)
            
            model_features[f'{group}_{num}'] = prediction
            
            del tokenizer, model; gc.collect()

In [ ]:
model_features_df = pd.DataFrame(model_features)
del model_features
model_features_df.sample(5)

In [ ]:
model_num = 0
preds = {}
for group in CFG.model_path.keys():

    filtered_features = [col for col in train_final.columns if group in col]
        
    scores = []
    y_pred = 0
    for n, clf in enumerate(clfs[group], start = 1):
        print(group, n)
        y_pred += clf.predict(model_features_df[filtered_features])#[:,1]
            
        
    y_pred /= n
    
    preds[group] = y_pred
    

In [ ]:
sub = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/sample_submission.csv')
sub['score'] = pd.DataFrame(preds).mean(axis=1).values
sub.to_csv('../working/submission.csv', index=False)

In [ ]:
!rm -rf ../working/catboost_info